In [7]:
from queue import PriorityQueue
from copy import deepcopy

SIDE = 3
puzzleSize = SIDE * SIDE   # 8-puzzle is 3 x 3

'''
The search node class
- state is a list representing the puzzle arrangement in row-major order. 
   0 represents blank. ex> [1, 2, 3, 8, 0, 4, 7, 6, 5]
- parent is a parent node
- operator is one of U, D, L, R
- evaluation value of node f = g + h
- moves record the path (sequence of actions)
'''
class Node:
    def __init__(self, state, parent, operator, g, h):
        self.state = state
        self.parent = parent
        self.operator = operator
        self.g = g
        self.h = h
        self.f = self.g + self.h
        if parent is None:
            self.moves = operator
        else:
            self.moves = parent.moves + '-' + operator
    def __lt__(self, other):
        return self.f < other.f

    '''
ASTAR performs A* search
- OPEN is a PriorityQueue of (node.f, node).
- CLOSED is a list of states
- When a next state is in OPEN, it should be updated.
   But instead, here we just put it into OPEN,
   and when it is chosen, we ignore the state if it is in CLOSED
xpand returns list of next (child) nodes
'''
def ASTAR(startNode):
    
    # initial OPEN and CLOSED
    OPEN = PriorityQueue()
    OPEN.put((startNode.f, startNode))
    CLOSED = []
    
    while not(OPEN.empty()):
        ##################################################################
        
        # Choose the best node from OPEN
        f, n = OPEN.get()
        # If it is in CLOSED, ignore
        if n not in CLOSED:
            CLOSED.append(n)    
            # Test if it is a goal
            if goaltest(n.state):
                return n
            # Expand the node
            else:
                nodeList = expand(n)
                for node in nodeList:
                    if node.state not in CLOSED:
                        OPEN.put((node.f, node))
        ##################################################################
                
    return None 
                
'''
goaltest returns 1 if state is same as the goalState
'''
def goaltest(state):
    if state == goalState:
        return 1
    else:
        return 0

'''
heuristic returns estimated distant from state to the goalState
'''
def heuristic(state):
    cnt= 0

    ##################################################################

    # compute h1 : number of tiles out of place 
    '''    
    for i in range(puzzleSize):
        if state[i] != goalState[i]:
            cnt += 1
    '''
    # compute h2 : sum of distances out of place
    for s_idx, s in enumerate(state):
        g_idx = goalState.index(s)
        sx, sy = s_idx // SIDE, s_idx % SIDE
        gx, gy = g_idx // SIDE, g_idx % SIDE
        cnt += (abs(gx-sx) + abs(gy-sy))   

    ##################################################################

    return cnt
'''
expand returns list of next (child) nodes
'''
def expand(node):
    nodeList = []
    state = node.state
    blank = state.index(0)   # location of blank
    row = int(blank / SIDE)
    col = blank % SIDE
    
    # up
    if not (row == 0):
        childState = deepcopy(state)
        childState[blank] = childState[blank-SIDE]
        childState[blank-SIDE] = 0
        nodeList.append(Node(childState, node, "U", node.g+1, heuristic(childState)))
    # down
    if not (row == SIDE-1):
        childState = deepcopy(state)
        childState[blank] = childState[blank+SIDE]
        childState[blank+SIDE] = 0
        nodeList.append(Node(childState, node, "D", node.g+1, heuristic(childState)))
    # left
    if not (col == 0):
        childState = deepcopy(state)
        childState[blank] = childState[blank-1]
        childState[blank-1] = 0
        nodeList.append(Node(childState, node, "L", node.g+1, heuristic(childState)))
    # right
    if not (col == SIDE-1):
        childState = deepcopy(state)
        childState[blank] = childState[blank+1]
        childState[blank+1] = 0
        nodeList.append(Node(childState, node, "R", node.g+1, heuristic(childState)))

    return nodeList

'''
printState print 3x3 puzzle state
'''
def printState(state):
    for i in range(3):
        for j in range(3):
            print(state[i*3 + j], end="\t")
        print("")
    print("")
    
'''
printSolutionPath show solution path recursively
'''
def printSolutionPath(node):
    if (node == None): return
    printSolutionPath(node.parent)
    printState(node.state)


In [8]:
'''
The start and goal states
'''
startState = [1, 2, 3, 
              8, 0, 4, 
              7, 6, 5]
goalState = [7, 8, 1,
             6, 0, 2,
             5, 4, 3]

print("Start state")
printState(startState)
print("Goal state")
printState(goalState)

'''
Make start node and do A* search
'''
startNode = Node(startState, None, "Start", 0, heuristic(startState))
print("Start A* search")
goalNode = ASTAR(startNode)
print(" Finished")

'''
If succeeded, show the solution path
'''
if (goalNode == None):
    print("A* Failed ")
else:
    print("A* Succeeded ")
    print("solution path: ", goalNode.moves)
    printSolutionPath(goalNode)


Start state
1	2	3	
8	0	4	
7	6	5	

Goal state
7	8	1	
6	0	2	
5	4	3	

Start A* search
 Finished
A* Succeeded 
solution path:  Start-L-D-R-R-U-U-L-L-D-D-R-R-U-U-L-D
1	2	3	
8	0	4	
7	6	5	

1	2	3	
0	8	4	
7	6	5	

1	2	3	
7	8	4	
0	6	5	

1	2	3	
7	8	4	
6	0	5	

1	2	3	
7	8	4	
6	5	0	

1	2	3	
7	8	0	
6	5	4	

1	2	0	
7	8	3	
6	5	4	

1	0	2	
7	8	3	
6	5	4	

0	1	2	
7	8	3	
6	5	4	

7	1	2	
0	8	3	
6	5	4	

7	1	2	
6	8	3	
0	5	4	

7	1	2	
6	8	3	
5	0	4	

7	1	2	
6	8	3	
5	4	0	

7	1	2	
6	8	0	
5	4	3	

7	1	0	
6	8	2	
5	4	3	

7	0	1	
6	8	2	
5	4	3	

7	8	1	
6	0	2	
5	4	3	

